In [1]:
#import os

In [2]:
#zip_file_path='spider_data.zip'

In [3]:
# extracted_dir = 'spider_dataset_extracted'
# os.makedirs(extracted_dir, exist_ok=True) 

In [4]:
# print(f"Extracting {zip_file_path} to {extracted_dir}...")
# !unzip -q {zip_file_path} -d {extracted_dir}

Extracting spider_data.zip to spider_dataset_extracted...


In [6]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset # Import Dataset from here
import json
import os

In [2]:

# --- 1. Configuration ---
MODEL_NAME = "t5-small" # You can try "t5-base" if you have more VRAM
TOKENIZER = T5Tokenizer.from_pretrained(MODEL_NAME)

# --- IMPORTANT: Your DATASET_DIR is correct now ---
# This should be the path to the 'spider_data' folder
DATASET_DIR = "/home/jaggu/Deep_L/spider_dataset_extracted/spider_data"

# Verify the directory exists
if not os.path.isdir(DATASET_DIR):
    raise FileNotFoundError(f"DATASET_DIR '{DATASET_DIR}' does not exist. Please check your path.")
# --- CORRECTED FILENAME CHECK ---
if not os.path.exists(os.path.join(DATASET_DIR, "train_spider.json")): # <<< CHANGED from train.json
    raise FileNotFoundError(f"train_spider.json not found in '{DATASET_DIR}'. Ensure Spider files are extracted correctly.")
if not os.path.exists(os.path.join(DATASET_DIR, "dev.json")):
    raise FileNotFoundError(f"dev.json not found in '{DATASET_DIR}'. Ensure Spider files are extracted correctly.")
if not os.path.exists(os.path.join(DATASET_DIR, "tables.json")):
    raise FileNotFoundError(f"tables.json not found in '{DATASET_DIR}'. Ensure Spider files are extracted correctly.")


# --- 2. Load tables.json for Database Schemas ---
tables_file_path = os.path.join(DATASET_DIR, 'tables.json')
with open(tables_file_path, 'r', encoding='utf-8') as f:
    db_schemas = {db['db_id']: db for db in json.load(f)}

print(f"Successfully loaded {len(db_schemas)} database schemas from {tables_file_path}")


Successfully loaded 166 database schemas from /home/jaggu/Deep_L/spider_dataset_extracted/spider_data/tables.json


In [7]:

def load_and_normalize_spider_split(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Prepare lists to hold normalized data for Dataset.from_dict
    questions = []
    queries = [] # This will hold the SQL strings
    db_ids = []

    for item in data:
        # Ensure 'query' is always a string.
        # Spider's dev.json and train_spider.json typically have 'query' as a string.
        # If it's a list (e.g., from train_others.json or other Spider variants),
        # you might need to decide how to handle it (e.g., take the first one, join them).
        # For standard Spider, 'query' should be a string.
        sql_query = item['query']
        if isinstance(sql_query, list):
            # This case shouldn't happen for standard train_spider.json/dev.json
            # but this handles the "cannot mix list and non-list" error.
            # We'll join them, or take the first one. Let's take the first for simplicity.
            sql_query = sql_query[0] if sql_query else ""
            print(f"Warning: Found list in 'query' field. Taking first element: {item['query']}")
        
        # Ensure all fields are present (though Spider typically is clean)
        if 'question' in item and 'db_id' in item and sql_query is not None:
            questions.append(item['question'])
            queries.append(sql_query)
            db_ids.append(item['db_id'])
        else:
            print(f"Warning: Skipping incomplete item: {item.get('question', 'N/A')}")

    # Create a dictionary suitable for datasets.Dataset.from_dict
    return Dataset.from_dict({
        'question': questions,
        'query': queries,
        'db_id': db_ids
    })

print("\nLoading and normalizing Spider train split...")
train_spider_dataset = load_and_normalize_spider_split(os.path.join(DATASET_DIR, 'train_spider.json'))
print("Loading and normalizing Spider validation split...")
validation_spider_dataset = load_and_normalize_spider_split(os.path.join(DATASET_DIR, 'dev.json'))


# Now assign to the spider_dataset dictionary as expected by the next steps
spider_dataset = {
    'train': train_spider_dataset,
    'validation': validation_spider_dataset
}


print("\nSpider Dataset loaded and normalized successfully!")
print(spider_dataset)



Loading and normalizing Spider train split...
Loading and normalizing Spider validation split...

Spider Dataset loaded and normalized successfully!
{'train': Dataset({
    features: ['question', 'query', 'db_id'],
    num_rows: 7000
}), 'validation': Dataset({
    features: ['question', 'query', 'db_id'],
    num_rows: 1034
})}


In [8]:

# --- 4. Schema Representation Function ---
def get_schema_representation(db_id, db_schemas_dict):
    """
    Generates a textual representation of the database schema for a given db_id.
    Includes table names, column names, and their types.
    """
    schema = db_schemas_dict[db_id]
    schema_parts = []
    
    for table_idx, table_name_original in enumerate(schema['table_names_original']):
        schema_parts.append(f"table {table_idx}: {table_name_original}")
        
        table_cols = []
        for col_idx, (col_table_idx, col_name_original) in enumerate(schema['column_names_original']):
            if col_table_idx == table_idx:
                col_type = schema['column_types'][col_idx]
                table_cols.append(f"column {col_idx}: {col_name_original} ({col_type})")
        
        if table_cols:
            schema_parts.append("  " + "; ".join(table_cols))
            
    return " | ".join(schema_parts)



In [9]:
# --- 5. Preprocessing Function for T5 (Tokenization) ---
def preprocess_function(examples):
    """
    Prepares input text for T5 model by combining schema and question,
    and tokenizes both input and target SQL queries.
    """
    inputs = []
    targets = []

    for i in range(len(examples['question'])):
        question = examples['question'][i]
        query = examples['query'][i]
        db_id = examples['db_id'][i]

        schema_text = get_schema_representation(db_id, db_schemas)

        input_text = f"generate sql: {schema_text} | question: {question}"
        
        inputs.append(input_text)
        targets.append(query)

    model_inputs = TOKENIZER(inputs, max_length=512, truncation=True, padding="max_length")
    labels = TOKENIZER(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
print("\nApplying preprocessing to the dataset (this may take a few minutes)...")
tokenized_train_dataset = spider_dataset['train'].map(preprocess_function, batched=True, remove_columns=['question', 'query', 'db_id'])
tokenized_eval_dataset = spider_dataset['validation'].map(preprocess_function, batched=True, remove_columns=['question', 'query', 'db_id'])
print("Dataset tokenization complete!")


Applying preprocessing to the dataset (this may take a few minutes)...


Map: 100%|█████████████████████████| 1034/1034 [00:00<00:00, 1497.99 examples/s]

Dataset tokenization complete!


In [11]:

# --- 6. Load Model ---
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
print(f"\nModel '{MODEL_NAME}' loaded.")


Model 't5-small' loaded.


In [14]:
# --- 7. Define Training Arguments ---
training_args = TrainingArguments(
    output_dir="./spider_t5_results", # Directory to save checkpoints and logs
    num_train_epochs=5,               # Keep 10 epochs (or more if eval loss keeps decreasing)
                                       # Adjust based on observation of eval_loss
    
    # --- ADJUSTED FOR 6GB VRAM ---
    per_device_train_batch_size=2,     # Reduced from 4 to 2. This is often the max for 6GB with long sequences.
    per_device_eval_batch_size=2,      # Reduced from 4 to 2 for consistency during evaluation.
    gradient_accumulation_steps=2,     # Increased from 1 to 2. This makes the *effective* batch size 2 * 2 = 4.
                                       # If you still get OOM, try:
                                       # per_device_train_batch_size=1, gradient_accumulation_steps=4
                                       # (This keeps effective batch size at 4 but uses even less peak VRAM)
    
    warmup_steps=200,                  # Keep 200
    weight_decay=0.01,                 # Keep as is
    learning_rate=5e-5,                # Keep 5e-5
    logging_dir="./spider_t5_logs",    # Directory for TensorBoard logs
    logging_steps=50,                  # Log training metrics every 50 steps
    eval_strategy="steps",       # Evaluate model performance every 'eval_steps'
    eval_steps=200,                    # Perform evaluation every 200 steps
    save_strategy="steps",             # Save a model checkpoint every 'save_steps'
    save_steps=200,
    load_best_model_at_end=True,       # After training, load the model with the best eval_loss
    metric_for_best_model="eval_loss", # Metric to monitor for saving the best model
    report_to="none",                  # Disable reporting to services like Weights & Biases if not using
    fp16=True,                         # ABSOLUTELY Keep this enabled. It's crucial for 6GB VRAM.
    # save_total_limit=3, # Keeps only the best 3 checkpoints (optional)
)

In [15]:
# --- 8. Create Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=TOKENIZER, # Pass tokenizer here, it's used for data collation and generation
)

/tmp/ipykernel_41766/2507042305.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
# --- 9. Train the Model ---
print("\nStarting training on Spider dataset...")
try:
    trainer.train()
    print("Training finished successfully!")
except RuntimeError as e:
    print(f"\nTraining interrupted due to RuntimeError: {e}")
    print("This often means Out-of-Memory (OOM). Try reducing 'per_device_train_batch_size' or 'max_length'.")


# --- 10. Save the fine-tuned model ---
model_save_path = "./fine_tuned_t5_spider_sql_generator"
model.save_pretrained(model_save_path)
TOKENIZER.save_pretrained(model_save_path)
print(f"\nFine-tuned model and tokenizer saved to {model_save_path}")


Starting training on Spider dataset...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
200,0.367400,0.254079
400,0.171200,0.136297
600,0.137600,0.113985
800,0.106500,0.103104
1000,0.105400,0.096909
1200,0.105500,0.093514
1400,0.094300,0.095755
1600,0.082800,0.089736
1800,0.080700,0.104546
2000,0.084500,0.093790


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Training finished successfully!

Fine-tuned model and tokenizer saved to ./fine_tuned_t5_spider_sql_generator
